In [163]:
import requests
import json
import dateutil.parser
import pandas as pd
import datetime

In [164]:
url = 'https://api.twitter.com/2/'
tweet_params = 'tweets/search/recent?max_results=100&tweet.fields=lang,created_at,public_metrics,entities&expansions=author_id&query=(fame OR family OR families OR %23family OR %23fam) (has:images OR has:videos) lang:en  -is:retweet'
user_params = '?user.fields=location,description,profile_image_url,public_metrics,verified'
bearer_token = 'AAAAAAAAAAAAAAAAAAAAABuMYgEAAAAAWbvBiTXJazZhs5Rym1SDJP9Le%2F8%3DXOz9r9sYN8VIhxv3l9qbkmDBhz3gDDyMSz3xN4dpCTQIhxrp0z'
headers = {'Authorization': 'Bearer ' + bearer_token}

In [165]:
def get_tweets(st, et):
    r = requests.get(url + tweet_params + '&start_time=' + st + '&end_time=' + et, headers=headers)
    res = json.loads(r.text)
    tweets = []
    for tweet in res['data']:
        data = {}
        data['id'] = tweet['id']
        data['text'] = tweet['text']
        iso_date = dateutil.parser.parse(tweet['created_at'])
        data['dayOfWeek'] = iso_date.isoweekday()
        data['time'] = iso_date.time()
        data['lang'] = tweet['lang']
        data['retweet_count'] = tweet['public_metrics']['retweet_count']
        data['like_count'] = tweet['public_metrics']['like_count']
        data['reply_count'] = tweet['public_metrics']['reply_count']
        data['quote_count'] = tweet['public_metrics']['quote_count']
        #! rerun and ensure cashtags is working
        ctags = []
        try:
            [tag['tag'] for tag in tweet['entities']['cashtags']]
        except: 
            pass
        data['cashtags'] = ctags
        tags = []
        try: 
            tags = [tag['tag'] for tag in tweet['entities']['hashtags']]
        except: 
            pass
        data['hashtags'] = tags
        len = 0
        try: 
            len = len(tweet['entities']['mentions']) 
        except:
            pass
        data['mention_count'] = len
        images = []
        try:
            images = [url['url'] for url in tweet['entities']['urls']]
        except:
            pass
        data['images'] = images
        data['author_id'] = tweet['author_id']
        tweets.append(data)

    #* I am exceeding the rate limit of 300/15 minutes. I need to rewrite this to return additional user data with a 15.1 minute sleep time between requests.
    # for tweet in tweets:
    #     ids = [t['author_id'] for t in tweets]
    #     r = requests.get(url +'users/' + id + user_params, headers=headers)
    #     res = json.loads(r.text)
    #     for prop in res['data']:
    #         tweet[prop] = res['data'][prop]

    return tweets
    

def iso(time):
    return time.isoformat()[:-3] + 'Z'

In [167]:
et = datetime.datetime.now()
st = (datetime.datetime.now() - datetime.timedelta(days=1))
print(st)
df = pd.DataFrame(get_tweets(iso(st), iso(et)))
#? set index to prevent duplicates?
while (df.index.size < 500):
    et = (et - datetime.timedelta(days=1))
    st = (st - datetime.timedelta(days=1))
    new_df = pd.DataFrame(get_tweets(iso(st), iso(et)))
    for i in new_df.index:
        print(new_df.id.iloc[i])
    df.append(new_df, ignore_index=True)

    

2022-01-28 23:54:45.200021
1487212576421392393
1487212569404215301
1487212561238052864
1487212526215516163
1487212521987604483
1487212521274724360
1487212510738620421
1487212506493820932
1487212500483457033
1487212493080608768
1487212488445902859
1487212481214918656
1487212475749707782
1487212466517934081
1487212462227267588
1487212454166040576
1487212425761992704
1487212423757336576
1487212420577861634
1487212418992414721
1487212416769437699
1487212413279776780
1487212412939948032
1487212394917019655
1487212381184925699
1487212375287676928
1487212370321711108
1487212367804985347
1487212361941454849
1487212361270566912
1487212350306500611
1487212344849707013
1487212344832892932
1487212334732890112
1487212325316841477
1487212294136238082
1487212294127988740
1487212274720944130
1487212267628335104
1487212265220804610
1487212249030791170
1487212245281038336
1487212228273229833
1487212222132670469
1487212217376264193
1487212208949972999
1487212203644174340
1487212184551763983
1487212182714

KeyError: 'data'

In [168]:
df

,id,text,dayOfWeek,time,lang,retweet_count,like_count,reply_count,quote_count,cashtags,hashtags,mention_count,images,author_id
0,1487574965260099588,@pulte This would go a long way to help my fam...,6,23:54:44,en,0,0,2,0,[],[],0,[https://t.co/K6wL3h0Tzk],25189181
1,1487574948768014336,whole family went out so i am chilling with hi...,6,23:54:40,en,0,0,0,0,[],[],0,[https://t.co/dZWzDnIJDQ],353484475
2,1487574939741958149,My thing is raising animals and plants on our ...,6,23:54:38,en,0,0,0,0,[],"[YouDoYourThing, Contest]",0,[https://t.co/0v3Ek5f40i],25192844
3,1487574937187635202,Stuffed but happy to see my family my other si...,6,23:54:37,en,0,7,0,0,[],[],0,[https://t.co/Qlpnb9Wtmf],4176577137
4,1487574883718643716,@LUMBERJACKJ0HN @bone_ape #WAGGA BEST FAMILY O...,6,23:54:24,en,0,1,1,0,[],[WAGGA],0,[https://t.co/wg2Jbzfgz8],1349271233884528640
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1487573843883573251,Finally - open the door to a place you and you...,6,23:50:16,en,0,0,0,0,[],"[newhome, homesforsale]",0,"[https://t.co/LE9XtXOmUG, https://t.co/WOFnfpj...",1051805597249744897
96,1487573841169760258,Not posting anything on my timeline about Tom ...,6,23:50:16,en,1,14,3,0,[],[Brady],0,"[https://t.co/6o6Zr7L2KC, https://t.co/6o6Zr7L...",546961396
97,1487573805610504198,I am excited to announce that I have committed...,6,23:50:07,en,5,96,18,8,[],[],0,[https://t.co/cKHHfcTxOy],971544137794248706
98,1487573790049529858,In a New World/Law in Order my beautiful yet S...,6,23:50:03,en,1,2,0,0,[],"[Insects, Animals, Fish]",0,"[https://t.co/GlfFyLlkpd, https://t.co/GlfFyLl...",1050125829114609667


In [ ]:
df.to_csv('tweets.csv')